In [ ]:
# install block2 and renormalizer using pip

Sometimes one may need to handle the overlap between a good DMRG solution from blaock2 and some external dense MPS, eg. Gaussian MPS that is useful for quantum information science or in [QMC calculations](https://arxiv.org/abs/2405.05440), or MPS from other DMRG codes, eg. [Renormalizer](https://github.com/shuaigroup/Renormalizer)

In [79]:
import numpy as np
from pyblock2._pyscf.ao2mo import integrals as itg
from pyblock2.driver.core import DMRGDriver, SymmetryTypes
from pyblock2.algebra.io import MPSTools
from pyblock2.algebra.core import SubTensor, Tensor
import numpy

bond_dims = [20] * 4 + [30] * 4
noises = [1e-4] * 4 + [1e-5] * 4 + [0]
thrds = [1e-10] * 8

from pyscf import gto, tools, lo
import tempfile
from pyscf.tools.fcidump import read
read_fcidump = read
r0 = 3.0
mol = gto.M(atom=f'H 0 0 0; H 0 0 {r0}; H 0 0 {r0*2}; H 0 0 {r0*3}; H 0 0 {r0*4}; H 0 0 {r0*5}', basis='sto-3g', unit='Bohr')
mol.build()
norb = mol.nao
S = mol.intor("int1e_ovlp_sph")
ao_coeff = lo.orth.lowdin(S)
ftmp = tempfile.NamedTemporaryFile()
tools.fcidump.from_mo(mol, ftmp.name, ao_coeff)
intg = read_fcidump(ftmp.name, norb)
h1e = intg['H1']
h2e = intg['H2']
ecore = intg['ECORE']

nelec = sum(mol.nelec)
spin = 0

# Run DMRG calculation with Sz symmetry
driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SZ)
driver.initialize_system(n_sites=norb, n_elec=nelec, spin=spin)
mpo = driver.get_qc_mpo(h1e=h1e, g2e=h2e, ecore=ecore, iprint=1)
zket = driver.get_random_mps(tag="KETSZ", bond_dim=20, nroots=1)
energies = driver.dmrg(mpo, zket, n_sweeps=20, bond_dims=bond_dims, noises=noises, thrds=thrds, iprint=1)
zket = driver.adjust_mps(zket, dot=1)[0]
driver.align_mps_center(zket, ref=0)
print('DMRG calculation with Sz symmetry finished, energies:', energies)

# this will be used to fill the external MPS's tensors
pyzket = MPSTools.from_block2(zket)

Parsing /tmp/tmptyyl7mo6
integral symmetrize error =  0.0
integral cutoff error =  0.0
mpo terms =       2286

Build MPO | Nsites =     6 | Nterms =       2286 | Algorithm = FastBIP | Cutoff = 1.00e-20
 Site =     0 /     6 .. Mmpo =    26 DW = 0.00e+00 NNZ =       26 SPT = 0.0000 Tmvc = 0.000 T = 0.002
 Site =     1 /     6 .. Mmpo =    66 DW = 0.00e+00 NNZ =      243 SPT = 0.8584 Tmvc = 0.000 T = 0.002
 Site =     2 /     6 .. Mmpo =   110 DW = 0.00e+00 NNZ =      459 SPT = 0.9368 Tmvc = 0.000 T = 0.003
 Site =     3 /     6 .. Mmpo =    66 DW = 0.00e+00 NNZ =     1147 SPT = 0.8420 Tmvc = 0.000 T = 0.003
 Site =     4 /     6 .. Mmpo =    26 DW = 0.00e+00 NNZ =      243 SPT = 0.8584 Tmvc = 0.000 T = 0.001
 Site =     5 /     6 .. Mmpo =     1 DW = 0.00e+00 NNZ =       26 SPT = 0.0000 Tmvc = 0.000 T = 0.001
Ttotal =      0.012 Tmvc-total = 0.002 MPO bond dimension =   110 MaxDW = 0.00e+00
NNZ =         2144 SIZE =        18004 SPT = 0.8809

Rank =     0 Ttotal =      0.025 MPO method 

here we also run a DMRG calculation with SU2 symmetry, and later we also compute its overlap calculation with the external MPS's tensors

In [80]:
# Run DMRG calculation with SU2 symmetry
driver_SU2 = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SU2)
driver_SU2.initialize_system(n_sites=norb, n_elec=nelec, spin=spin)
mpo = driver_SU2.get_qc_mpo(h1e=h1e, g2e=h2e, ecore=ecore, iprint=1)
ket = driver_SU2.get_random_mps(tag="KETSU2", bond_dim=20, nroots=1)
energies = driver_SU2.dmrg(mpo, ket, n_sweeps=20, bond_dims=bond_dims, noises=noises, thrds=thrds, iprint=1)
ket = driver_SU2.adjust_mps(ket, dot=1)[0]
driver_SU2.align_mps_center(ket, ref=0)
print('DMRG calculation with SU2 symmetry finished, energies:', energies)

integral symmetrize error =  0.0
integral cutoff error =  0.0
mpo terms =        863

Build MPO | Nsites =     6 | Nterms =        863 | Algorithm = FastBIP | Cutoff = 1.00e-20
 Site =     0 /     6 .. Mmpo =    13 DW = 0.00e+00 NNZ =       13 SPT = 0.0000 Tmvc = 0.000 T = 0.002
 Site =     1 /     6 .. Mmpo =    34 DW = 0.00e+00 NNZ =       96 SPT = 0.7828 Tmvc = 0.000 T = 0.002
 Site =     2 /     6 .. Mmpo =    56 DW = 0.00e+00 NNZ =      180 SPT = 0.9055 Tmvc = 0.000 T = 0.002
 Site =     3 /     6 .. Mmpo =    34 DW = 0.00e+00 NNZ =      406 SPT = 0.7868 Tmvc = 0.000 T = 0.002
 Site =     4 /     6 .. Mmpo =    14 DW = 0.00e+00 NNZ =       99 SPT = 0.7920 Tmvc = 0.000 T = 0.001
 Site =     5 /     6 .. Mmpo =     1 DW = 0.00e+00 NNZ =       14 SPT = 0.0000 Tmvc = 0.000 T = 0.001


Ttotal =      0.009 Tmvc-total = 0.001 MPO bond dimension =    56 MaxDW = 0.00e+00
NNZ =          808 SIZE =         4753 SPT = 0.8300

Rank =     0 Ttotal =      0.016 MPO method = FastBipartite bond dimension =      56 NNZ =          808 SIZE =         4753 SPT = 0.8300

Sweep =    0 | Direction =  forward | Bond dimension =   20 | Noise =  1.00e-04 | Dav threshold =  1.00e-10
Time elapsed =      0.098 | E =      -2.9576460854 | DW = 1.81841e-06

Sweep =    1 | Direction = backward | Bond dimension =   20 | Noise =  1.00e-04 | Dav threshold =  1.00e-10
Time elapsed =      0.188 | E =      -2.9576460854 | DE = 8.52e-12 | DW = 9.59940e-07

Sweep =    2 | Direction =  forward | Bond dimension =   20 | Noise =  1.00e-04 | Dav threshold =  1.00e-10
Time elapsed =      0.285 | E =      -2.9576460854 | DE = -3.08e-12 | DW = 1.81905e-06

Sweep =    3 | Direction = backward | Bond dimension =   20 | Noise =  1.00e-04 | Dav threshold =  1.00e-10
Time elapsed =      0.374 | E =      -2.95764608

As mentioned, Renormalizer runs DMRG with Sz symmetry by default, and the MPS object is a dense MPS. So below we obtain such an MPS object from Renormalizer calculations as an example.


In [81]:
from renormalizer import Model, Mps, Mpo, optimize_mps
from renormalizer.model import h_qc
from renormalizer.utils import log

logger = logging.getLogger("renormalizer")

h1e_spin, h2e_spin, nuc = h_qc.read_fcidump(ftmp.name, norb)
basis, ham_terms = h_qc.qc_model(h1e_spin, h2e_spin)
model = Model(basis, ham_terms)
mpo = Mpo(model)
logger.info(f"mpo_bond_dims:{mpo.bond_dims}")

energy_list = {}
M = 30
procedure = [[M, 0.4], [M, 0.2], [M, 0.1], [M, 0], [M, 0], [M,0], [M,0]]
mps = Mps.random(model, mol.nelec, M, percent=1.0)
mps.optimize_config.procedure = procedure
mps.optimize_config.method = "2site"
energies, mps = optimize_mps(mps.copy(), mpo)
gs_e = min(energies)+nuc
logger.info(f"lowest energy: {gs_e}")

2025-06-13 20:58:23,638[INFO] nuclear repulsion: 2.899999999999999
2025-06-13 20:58:23,639[INFO] spin norbs: 12


2025-06-13 20:58:24,462[DEBUG] # of operator terms: 1818
2025-06-13 20:58:24,463[DEBUG] symbolic mpo algorithm: Hopcroft-Karp
2025-06-13 20:58:24,463[DEBUG] Input operator terms: 1818
2025-06-13 20:58:24,711[DEBUG] After combination of the same terms: 1818
2025-06-13 20:58:24,947[INFO] mpo_bond_dims:[1, 4, 16, 39, 54, 71, 92, 71, 54, 39, 16, 4, 1]
2025-06-13 20:58:24,977[INFO] optimization method: 2site
2025-06-13 20:58:24,977[INFO] e_rtol: 1e-06
2025-06-13 20:58:24,978[INFO] e_atol: 1e-08
2025-06-13 20:58:24,978[INFO] procedure: [[30, 0.4], [30, 0.2], [30, 0.1], [30, 0], [30, 0], [30, 0], [30, 0]]
2025-06-13 20:58:24,996[DEBUG] isweep: 0
2025-06-13 20:58:24,996[DEBUG] compress config in current loop: 30, percent: 0.4
2025-06-13 20:58:24,997[DEBUG] mps current size: 39.8KiB, Matrix product bond dim:[1, 2, 4, 8, 16, 30, 29, 21, 13, 8, 4, 2, 1]
2025-06-13 20:58:24,997[DEBUG] optimize site: [0, 1]
2025-06-13 20:58:24,998[DEBUG] use direct eigensolver
2025-06-13 20:58:24,999[DEBUG] energy:

Note that now we obtain a dense MPS in the spin orbital basis, to make it compatible with block2, we need to convert it to a fermion orbital basis.
We also need to make the MPS left canonical, for the step after.

In [82]:
mps.ensure_left_canonical()

def spin_to_fermion_mps(mps_spin, qn):
    nsites_spin = len(mps_spin)
    assert nsites_spin % 2 == 0
    nsites_fermion = nsites_spin // 2
    qnl_fermion = []
    mps_fermion = []
    
    for i in range(nsites_fermion):
        ml = mps_spin[2*i].shape[0]
        mr = mps_spin[2*i+1].shape[-1]
        merged_array = numpy.einsum("ipj, jql->ipql", mps_spin[2*i], mps_spin[2*i+1])
        merged_array = numpy.transpose(merged_array, (0, 2, 1, 3))
        merged_array = merged_array.reshape(ml, 4, mr)
        mps_fermion.append(merged_array)
        qnl_fermion.append(qn[2*i])
    qnl_fermion.append(qn[-1])
    return mps_fermion, qnl_fermion

qn = []
for i_mps in range(len(mps)):
    iqn = [x.tolist() for x in mps.qn[i_mps]]
    qn.append(iqn)
qn[0] = [[0, 0]]    
qn.append([[mol.nelec[0], mol.nelec[1]]])

mps_fermion, qn_fermion = spin_to_fermion_mps(mps, qn)
mps_list = []
for i_mps in range(len(mps_fermion)):
    mps_list.append(mps_fermion[i_mps])

Now we have a list of dense MPS tensors in the fermion orbital basis, and a list of their quantum numbers (from left to right). And we are ready to transform them to block2 format.

In [ ]:
print('bond dimensions:', [imps.shape[0] for imps in mps_fermion])
print('quantum numbers:', qn_fermion)

from pyblock2.algebra.io import MPSTools
transform_renormalizer_sz_to_block2_sz = MPSTools.trans_dense_sz_to_block2_sz
mps_in_block2 = transform_renormalizer_sz_to_block2_sz(mps_list, qn_fermion)

bond dimensions: [1, 4, 16, 30, 16, 4]
quantum numbers: [[[0, 0]], [[1, 0], [0, 1], [1, 1], [0, 0]], [[0, 1], [0, 1], [1, 2], [1, 2], [2, 1], [2, 1], [0, 0], [1, 1], [1, 1], [1, 1], [1, 1], [2, 0], [0, 2], [2, 2], [1, 0], [1, 0]], [[0, 1], [1, 2], [1, 2], [1, 2], [1, 2], [2, 1], [2, 1], [2, 1], [2, 1], [3, 1], [3, 1], [1, 1], [1, 1], [1, 1], [1, 1], [0, 3], [2, 0], [2, 0], [3, 0], [2, 3], [0, 2], [0, 2], [2, 2], [2, 2], [2, 2], [2, 2], [1, 0], [3, 2], [1, 3], [1, 3]], [[1, 2], [1, 2], [2, 1], [2, 1], [3, 1], [1, 1], [2, 3], [2, 3], [3, 3], [2, 2], [2, 2], [2, 2], [2, 2], [3, 2], [3, 2], [1, 3]], [[2, 3], [3, 3], [2, 2], [3, 2]], [[3, 3]]]


Now MPS_in_block2 contains all the tensors in the pyblock2 format, now further convert it to block2 format.

In [84]:
driver.symm_type = SymmetryTypes.SZ
driver.initialize_system(n_sites=norb, n_elec=nelec, spin=spin)
impo = driver.get_identity_mpo()
for it, ts in enumerate(pyzket.tensors):
    pyzket.tensors[it] = mps_in_block2[it]
bra = MPSTools.to_block2(pyzket, driver.basis, tag='bra_sz')
print('overlap between Renormalizer Sz MPS with block2 Sz MPS:', driver.expectation(bra, impo, zket))

overlap between Renormalizer Sz MPS with block2 Sz MPS: 0.9999999659592702


We also want to compute the overlap between the Renormalizer Sz MPS and the block2 SU2 MPS. To do this, we first need to convert the Renormalizer Sz MPS to a SU2 MPS.

In [85]:
driver.symm_type = SymmetryTypes.SZ
driver.initialize_system(n_sites=norb, n_elec=nelec, spin=spin)
for it, ts in enumerate(pyzket.tensors):
    pyzket.tensors[it] = mps_in_block2[it]
bra = MPSTools.trans_sz_to_su2(pyzket, driver.basis, zket.info.target, target_twos=0)

driver.symm_type = SymmetryTypes.SU2
driver.initialize_system(n_sites=norb, n_elec=nelec, spin=spin)
bra = MPSTools.to_block2(bra, driver.basis, tag='bra_su2')

impo = driver.get_identity_mpo()
print('overlap between Renormalizer Sz MPS with block2 SU2 MPS:', driver.expectation(bra, impo, ket))

overlap between Renormalizer Sz MPS with block2 SU2 MPS: -0.9999990300149895


Note that different DMRG runs may lead to sign differences. 